# Homework 2
#### Pierre-Antoine Desplaces, Anaïs Ladoy, Lou Richard

In [62]:
# Import libraries
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [63]:
def bars(title, data, xlab, ylab):
    sns.set_style('darkgrid')
    fig, ax = plt.subplots(figsize = (15,8))
    ax.set_title(title, fontsize=15, fontweight='bold')
    sns.barplot(x=xlab, y=ylab, data=data, saturation=0.7, errcolor='.7')
    plt.xticks(rotation=90)
    plt.show()

## <b> <font color='purple'>Question 1</font> </b>
<b>Go to the eurostat website and try to find a dataset that includes the european unemployment rates at a recent date.

Use this data to build a Choropleth map which shows the unemployment rate in Europe at a country level. Think about the colors you use, how you decided to split the intervals into data classes or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.</b>

We find the unemployment information under Population and social conditions > Labour Market > Employment and unemployment. We chose to work with the Unemployment rates by sex, age and educational attainment level database found in the LFS - detailed annual survey results.

In [64]:
# Load the file
raw_euro = pd.read_csv('lfsa_urgaed_1_Data.csv')

Looking at the data, we see that we have a lot of unecessary informations. Since we just need the total unemployment rate, not considering the level of education nor the gender, we keep only the 'Total' rows from Sex column, and the 'All ISCED 2011 levels' from the ISCED11 column.

In [65]:
# Remove unecessary rows
raw_euro = raw_euro[raw_euro.SEX == 'Total']
raw_euro = raw_euro[raw_euro.ISCED11 == 'All ISCED 2011 levels ']

We now create the Dataframe we will use keeping only the relevant columns GEO and Value. We then convert our values into a float type.

In [66]:
# Create dataframe with only the relevant columns
df_euro = raw_euro[['GEO', 'Value']].copy()
df_euro.columns = ('Country', 'Unemployment (%)')
# Adjust the index
df_euro.set_index((df_euro.index/15).astype(int), inplace=True)
# Convert the values to float
df_euro['Unemployment (%)'] = df_euro['Unemployment (%)'].astype(float)

We now load the json we will use to display our map.

In [67]:
# Load the json data of the map
euro_geo_path = 'topojson/europe.topojson.json'
euro_geo = json.load(open(euro_geo_path))
# First element of the json
euro_geo['objects']['europe']['geometries'][0]

{'arcs': [[[0, 1, 2]], [[3]], [[4]], [[5, 6, 7, 8, 9, 10], [11]]],
 'id': 'AZ',
 'properties': {'NAME': 'Azerbaijan'},
 'type': 'MultiPolygon'}

When we look at the json, we see that we will have to use the country IDs to match the values from our dataframe to the geometries of the json. We thus create a dictionary mapping the country names to their IDs. We also change the name of Germany and Macedonia in our dataframe in order to match them with the json data. We then add a new column in our dataframe containing the IDs for each country.

In [68]:
# Clean the name of Germany and Macedonia
df_euro.Country = df_euro.Country.replace({'Germany (until 1990 former territory of the FRG)': 'Germany'})
df_euro.Country = df_euro.Country.replace({'Former Yugoslav Republic of Macedonia, the': 'The former Yugoslav Republic of Macedonia'})
# Create a dictionary to convert country names to country IDs
dict_country_id = dict(map(lambda x : (x['properties']['NAME'],x['id']),euro_geo['objects']['europe']['geometries']))
# Adding a new column with country IDs
df_euro['Country ID'] = df_euro.Country.map(lambda x : dict_country_id[x])
# Resulting dataframe
df_euro.head()

Country  Unemployment (%) Country ID
0         Belgium               7.8         BE
1        Bulgaria               7.6         BG
2  Czech Republic               4.0         CZ
3         Denmark               6.2         DK
4         Germany               4.1         DE

To get a better idea of the repartition of the rates, we first display the values in a bar charts.

In [69]:
bars("Unemployment in Europe", df_euro, 'Country ID', 'Unemployment (%)')

We now create our map. To define the threshold_scale, we use the mean and the standard deviation of our values. We divide them according to their distance to the mean in terms of the standard deviation.

In [70]:
# Construct the threshold 
mean = df_euro['Unemployment (%)'].mean()
std = df_euro['Unemployment (%)'].std()
min_rate = df_euro['Unemployment (%)'].min()
max_rate = df_euro['Unemployment (%)'].max()

def dist_std(x) :
    return mean + x*std

threshold = [min_rate,dist_std(-1),dist_std(0),dist_std(1),dist_std(2),max_rate]

# Display the visualization map
m_euro = folium.Map([54,15], zoom_start=4)
m_euro.choropleth(
    geo_data = euro_geo,
    data=df_euro,
    columns=['Country ID', 'Unemployment (%)'],
    key_on='feature.id',
    topojson='objects.europe',
    fill_color = 'OrRd', fill_opacity=0.5, line_opacity=0.2,
    legend_name="Unemployment in Europe (in % of population)",
    threshold_scale = threshold
    )
m_euro

The countries with no color are those for which we don't have unemployment informations from Eurostat.

Looking at the map, we observe that three countries stand out : Spain, Macedonia and Greece. They have the highest unemployment rate. Countries with high unemployment rate are mostly mediterranean countries : France, Italy, Turkey. 

If we look at Switzerland, we see that it belongs to countries with a quite low unemployment rate.

## <font color='purple'>Question 2</font> 
<b>Go to the amstat website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

HINT Go to the details tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through.
Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.</b>

The unemployement rates of swiss cantons are downloadable through the amstat website (https://www.amstat.ch/v2/index.jsp) according several options we can specify (specific period, geographic level, economic or social attributes, ...). We choose to download the latest data that were available, corresponding to the unemployement rates for September 2017 at a canton level.

In [71]:
amstat_unemp_rate=pd.read_csv('amstat_taux_chomage.csv',sep=',',skiprows=1,skipfooter=1,thousands="'",encoding='utf-16',engine='python')
amstat_unemp_rate.drop(amstat_unemp_rate.columns[[1, 9, 10, 11, 12, 13, 14,15]],axis=1,inplace=True)
amstat_unemp_rate.rename(columns={' ':'Canton'},inplace=True)

In [72]:
amstat_unemp_rate.dtypes

Canton                               object
Taux de chômage                     float64
Coefficients de variation            object
Chômeurs inscrits                     int64
Chômeurs jeunes                       int64
Chômeurs de longue durée              int64
Demandeurs d'emploi                   int64
Demandeurs d'emploi non chômeurs      int64
dtype: object

In [73]:
swiss_cantons=json.load(open('./topojson/ch-cantons.topojson.json'))
# Create a list with the corresponding code (id) for each canton
cantons_id=[(i['properties']['name'],i['id']) for i in swiss_cantons['objects']['cantons']['geometries']]

In [74]:
cantons_id=pd.DataFrame(cantons_id,columns=['Canton','Code'])
cantons_id

Canton Code
0                   Zürich   ZH
1               Bern/Berne   BE
2                   Luzern   LU
3                      Uri   UR
4                   Schwyz   SZ
5                 Obwalden   OW
6                Nidwalden   NW
7                   Glarus   GL
8                      Zug   ZG
9                 Fribourg   FR
10               Solothurn   SO
11             Basel-Stadt   BS
12        Basel-Landschaft   BL
13            Schaffhausen   SH
14  Appenzell Ausserrhoden   AR
15   Appenzell Innerrhoden   AI
16              St. Gallen   SG
17     Graubünden/Grigioni   GR
18                  Aargau   AG
19                 Thurgau   TG
20                  Ticino   TI
21                    Vaud   VD
22           Valais/Wallis   VS
23               Neuchâtel   NE
24                  Genève   GE
25                    Jura   JU

We can notice that the swiss cantons names in our dataframe and in the TopoJSON file are really different as the names in the TopoJSON are expressed in the official language for each canton. 
Fortunately, we can see that the cantons are in the same order in both our dataframe and the TopoJSON, we can thus match the canton code according the index.

In [75]:
# Adding a new column with country IDs
amstat_unemp_rate = amstat_unemp_rate.merge(cantons_id[['Code']],left_index=True,right_index=True)

The Swiss Confederation defines the unemployment rate as follow :

- Unemployment Rate = 100 x Job seekers / Active Population  (Taux de chômage=100xDemandeurs d'emploi/Population active)

Nevertheless, the job seekers include both unemployed people and people who are not immediately available for a new job (because they have already one or they actually follow training/upgrading programs).

Then, it could be interesting to compare the previous map with the proportion in each canton of unemployed people searching for a job.

- Unemployed Job Seekers Rate = Registered Unemployed * Unemployment Rate / Job Seekers

In [76]:
amstat_unemp_rate["Taux de demandeurs d'emplois chômeurs"]=amstat_unemp_rate['Chômeurs inscrits']*amstat_unemp_rate['Taux de chômage']/amstat_unemp_rate["Demandeurs d'emploi"]

In [77]:
amstat_unemp_rate

Canton  Taux de chômage Coefficients de variation  \
0                         Zurich              3.3                         A   
1                          Berne              2.4                         A   
2                        Lucerne              1.7                         A   
3                            Uri              0.6                         C   
4                         Schwyz              1.7                         A   
5                         Obwald              0.7                         B   
6                        Nidwald              1.0                         B   
7                         Glaris              1.8                         B   
8                           Zoug              2.3                         B   
9                       Fribourg              2.7                         A   
10                       Soleure              2.6                         A   
11                    Bâle-Ville              3.5                         A   
12                 Bâle-Campagne              2.8                         A   
13                   Schaffhouse              3.0                         B   
14  Appenzell Rhodes-Extérieures              1.7                         B   
15  Appenzell Rhodes-Intérieures              0.7                         C   
16                       St-Gall              2.2                         A   
17                       Grisons              1.1                         A   
18                       Argovie              2.9                         A   
19                     Thurgovie              2.0                         A   
20                        Tessin              3.1                         A   
21                          Vaud              4.3                         A   
22                        Valais              2.8                         A   
23                     Neuchâtel              5.1                         A   
24                        Genève              5.2                         A   
25                          Jura              4.4                         B   

    Chômeurs inscrits  Chômeurs jeunes  Chômeurs de longue durée  \
0               27225             3420                      4753   
1               13658             2003                      2336   
2                3885              664                       453   
3                 112               15                        13   
4                1455              185                       189   
5                 153               21                        10   
6                 248               31                        24   
7                 416               60                        78   
8                1543              133                       248   
9                4466              683                       486   
10               3801              584                       568   
11               3455              408                       606   
12               4082              524                       651   
13               1286              226                       268   
14                523               65                        95   
15                 62               16                         9   
16               6127             1046                       866   
17               1166              194                        64   
18              10684             1555                      1698   
19               3058              517                       416   
20               5202              803                       797   
21              17155             1904                      3015   
22               4816              746                       588   
23               4738              726                      1210   
24              12234              914                      2326   
25               1619              266                       446   

    Demandeurs d'emploi  Demandeurs d'emploi non chômeu

The first cloropleth map we visualize is the unemployment rate in the Swiss cantons for the month of September 2017 according the definition of the Swiss Confederation.

In [78]:
amstat_unemp1_map=folium.Map(location=[46.900000, 8.226667],tiles='cartodbpositron',zoom_start=8)
amstat_unemp1_map.choropleth(
geo_data=swiss_cantons,
data=amstat_unemp_rate,
columns=['Code', 'Taux de chômage'],
key_on='feature.id',
topojson='objects.cantons',
fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.4,line_color='white',
legend_name="Unemployment rate (Nb of job seekers/Nb of active persons in %)"
)
# Save the map in an HTML file
amstat_unemp1_map.save('Swiss_unemployment_rate_1.html')
amstat_unemp1_map

COMMENT

In [79]:
amstat_unemp2_map=folium.Map(location=[46.900000, 8.226667],tiles='cartodbpositron',zoom_start=8)
amstat_unemp2_map.choropleth(
geo_data=swiss_cantons,
data=amstat_unemp_rate,
columns=['Code', "Taux de demandeurs d'emplois chômeurs"],
key_on='feature.id',
topojson='objects.cantons',
fill_color='YlOrRd', fill_opacity=0.7, line_opacity=0.4,line_color='white',
legend_name="Unemployment rate without people having already a job"
)
# Save the map in an HTML file
amstat_unemp2_map.save('Swiss_unemployment_rate_2.html')
amstat_unemp2_map

In [ ]:
COMMENT

## <font color='purple'>Question 3</font> 
<b>Use the amstat website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between Swiss and foreign workers.

The Economic Secretary (SECO) releases a monthly report on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for foreign (5.1%) and Swiss (2.2%) workers.

Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (hint The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.</b>

## <font color='purple'>Question 4</font> 
<b>BONUS: using the map you have just built, and the geographical information contained in it, could you give a rough estimate of the difference in unemployment rates between the areas divided by the Röstigraben?</b>